In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta

In [1]:
!pwd

/Users/sushanta/Documents/GitHub/Trading


In [2]:
def load_data():
    #All the data loaded from the location /Users/sushanta/Documents/Sushanta/Share Market/data/
    df_bank = pd.read_csv("/Users/sushanta/Documents/Sushanta/Share Market/data/Bank.csv") #Banknifty Data 
    df_ce = pd.read_csv("/Users/sushanta/Documents/Sushanta/Share Market/data/CE.csv") # CE data
    df_pe = pd.read_csv("/Users/sushanta/Documents/Sushanta/Share Market/data/PE.csv") # PE data
    return df_bank,df_ce,df_pe

def derive_strike_price(curr_price):
    #Below code to extract which CE or PE strike price we need to go for
    extract_thousand = np.int(np.floor(curr_price/1000)*1000)
    extract_hundred = np.int((np.floor((curr_price - np.floor(curr_price/1000)*1000)/100))*100)
    extract_decimal = np.float(curr_price - (extract_thousand + extract_hundred))
    if extract_decimal > 70:
        strike_price = (extract_thousand + extract_hundred + 100)
    else:
        strike_price = (extract_thousand + extract_hundred)
        
    return strike_price

def trade_now(trade_date):
    curr_monday = trade_date #'07-Dec-20'
    points_collected = 0
    curr_monday_datetime = pd.to_datetime(curr_monday)
    expiry_thursday = (curr_monday_datetime + timedelta(days=3)).strftime("%d-%b-%y")
    bank_close_price_monday = np.float(df_bank[df_bank['Date'] == curr_monday]['Close'])
    adjust_ind = False
    adjusted_price = 0
    distance_percent = 1.6
    
    print ("Processing Date: {}".format(curr_monday))
    print ("Expiry Date    : {}".format(expiry_thursday))
    print ("Monday Price(C): {}".format(bank_close_price_monday))

    #Below code to extract which price we need to hit for
    bank_monday_ce_price = bank_close_price_monday + ((bank_close_price_monday*distance_percent)/100)
    bank_monday_pe_price = bank_close_price_monday - ((bank_close_price_monday*distance_percent)/100)

    #Below code to extract which CE or PE strike price we need to go for
    strike_price_ce = derive_strike_price(bank_monday_ce_price)
    strike_price_pe = derive_strike_price(bank_monday_pe_price)
    print ("Targeted CE (C): {}".format(strike_price_ce))
    print ("Targeted PE (C): {}".format(strike_price_pe))

    #Extract the Closing Price of the CE & PE (Monday)
    ce_buy_close_price_monday = np.float(df_ce[(df_ce['Strike Price'] == (strike_price_ce+300)) & (df_ce['Date'] == curr_monday) & (df_ce['Expiry'] == expiry_thursday)]['Close'])
    ce_sell_close_price_monday = np.float(df_ce[(df_ce['Strike Price'] == strike_price_ce) & (df_ce['Date'] == curr_monday) & (df_ce['Expiry'] == expiry_thursday)]['Close'])
    pe_sell_close_price_monday = np.float(df_pe[(df_pe['Strike Price'] == strike_price_pe) & (df_pe['Date'] == curr_monday) & (df_pe['Expiry'] == expiry_thursday)]['Close'])
    pe_buy_close_price_monday = np.float(df_pe[(df_pe['Strike Price'] == (strike_price_pe-300)) & (df_pe['Date'] == curr_monday) & (df_pe['Expiry'] == expiry_thursday)]['Close'])
    print (" -----------------")
    print ("|MONDAY|{}|".format(curr_monday))
    print (" -----------------")
    print ("Buy - {}CE : {:.1f}".format((strike_price_ce+300),ce_buy_close_price_monday))
    print ("Sell- {}CE : {:.1f}".format(strike_price_ce,ce_sell_close_price_monday))
    print ("Sell- {}PE : {:.1f}".format(strike_price_pe,pe_sell_close_price_monday))
    print ("Buy - {}PE : {:.1f}".format((strike_price_pe-300),pe_buy_close_price_monday))

    print ("-------------------")
    print ("Points Target : {:.1f}".format((ce_sell_close_price_monday -ce_buy_close_price_monday)+(pe_sell_close_price_monday-pe_buy_close_price_monday)))
    print ("Total Points  : {:.1f} <=".format(points_collected))
    #print ("----------------------------------------------------")

    #Go to the Next Day - Which is Tuesday
    next_day_tuesday = (curr_monday_datetime + timedelta(days=1)).strftime("%d-%b-%y")

    #Extract the Closing Price of the CE & PE (Tuesday)
    ce_buy_close_price_tuesday = np.float(df_ce[(df_ce['Strike Price'] == (strike_price_ce+300)) & (df_ce['Date'] == next_day_tuesday) & (df_ce['Expiry'] == expiry_thursday)]['Close'])
    ce_sell_close_price_tuesday = np.float(df_ce[(df_ce['Strike Price'] == strike_price_ce) & (df_ce['Date'] == next_day_tuesday) & (df_ce['Expiry'] == expiry_thursday)]['Close'])
    pe_sell_close_price_tuesday = np.float(df_pe[(df_pe['Strike Price'] == strike_price_pe) & (df_pe['Date'] == next_day_tuesday) & (df_pe['Expiry'] == expiry_thursday)]['Close'])
    pe_buy_close_price_tuesday = np.float(df_pe[(df_pe['Strike Price'] == (strike_price_pe-300)) & (df_pe['Date'] == next_day_tuesday) & (df_pe['Expiry'] == expiry_thursday)]['Close'])
    print (" -----------------")
    print ("|TUESDAY|{}|".format(next_day_tuesday))
    print (" -----------------")
    print ("Buy - {}CE : {:.1f}".format((strike_price_ce+300),ce_buy_close_price_tuesday))
    print ("Sell- {}CE : {:.1f}".format(strike_price_ce,ce_sell_close_price_tuesday))
    print ("Sell- {}PE : {:.1f}".format(strike_price_pe,pe_sell_close_price_tuesday))
    print ("Buy - {}PE : {:.1f}".format((strike_price_pe-300),pe_buy_close_price_tuesday))

    buy_ce = (ce_buy_close_price_tuesday - ce_buy_close_price_monday)
    sell_ce = (ce_sell_close_price_monday - ce_sell_close_price_tuesday)
    sell_pe = (pe_sell_close_price_monday - pe_sell_close_price_tuesday)
    buy_pe = (pe_buy_close_price_tuesday - pe_buy_close_price_monday)


    #It's time now to do adjustment
    if (ce_sell_close_price_tuesday > pe_sell_close_price_tuesday):
        if ( ((ce_sell_close_price_tuesday*30)/100) > pe_sell_close_price_tuesday):
            print ("************* Adjustment for CE *************")
            adjust_ind = True
            
            adjusted_price = sell_pe
            points_collected = adjusted_price
            
            #points_collected_tuesday = points_collected

            new_strike_price_pe = df_pe.iloc[df_pe[(df_pe['Expiry'] == expiry_thursday) & (df_pe['Date'] == next_day_tuesday) & (df_pe['Close'] >= (ce_sell_close_price_tuesday-15)) & (df_pe['Close'] <= (ce_sell_close_price_tuesday+15)) & (df_pe['Strike Price'] > strike_price_pe) ].index[0]]['Strike Price']
            #strike_price_pe = new_strike_price_pe

            ce_buy_close_price_tuesday = np.float(df_ce[(df_ce['Strike Price'] == (strike_price_ce+300)) & (df_ce['Date'] == next_day_tuesday) & (df_ce['Expiry'] == expiry_thursday)]['Close'])
            ce_sell_close_price_tuesday = np.float(df_ce[(df_ce['Strike Price'] == strike_price_ce) & (df_ce['Date'] == next_day_tuesday) & (df_ce['Expiry'] == expiry_thursday)]['Close'])
            pe_sell_close_price_tuesday = np.float(df_pe[(df_pe['Strike Price'] == new_strike_price_pe) & (df_pe['Date'] == next_day_tuesday) & (df_pe['Expiry'] == expiry_thursday)]['Close'])
            pe_buy_close_price_tuesday = np.float(df_pe[(df_pe['Strike Price'] == (strike_price_pe-300)) & (df_pe['Date'] == next_day_tuesday) & (df_pe['Expiry'] == expiry_thursday)]['Close'])

            print ("Buy - {}CE : {:.1f}".format((strike_price_ce+300),ce_buy_close_price_tuesday))
            print ("Sell- {}CE : {:.1f}".format(strike_price_ce,ce_sell_close_price_tuesday))
            print ("Sell- {}PE : {:.1f}".format(new_strike_price_pe,pe_sell_close_price_tuesday))
            print ("Buy - {}PE : {:.1f}".format((strike_price_pe-300),pe_buy_close_price_tuesday))

            buy_ce = (ce_buy_close_price_tuesday - ce_buy_close_price_monday)
            sell_ce = (ce_sell_close_price_monday - ce_sell_close_price_tuesday)
            sell_pe = (pe_sell_close_price_monday - pe_sell_close_price_tuesday)
            buy_pe = (pe_buy_close_price_tuesday - pe_buy_close_price_monday)
            
    if (pe_sell_close_price_tuesday > ce_sell_close_price_tuesday):
        if ( ((pe_sell_close_price_tuesday*30)/100) > ce_sell_close_price_tuesday):
            print ("************* Adjustment for PE *************")

    if (adjust_ind):
        #print ("In If Statement....{}".format(pe_sell_close_price_tuesday))
        print ("New Points     : {:.1f} <=".format(points_collected))
    else:
        new_strike_price_pe = strike_price_pe
        #print ("In Else Statement ...")
        points_collected_tuesday = ((buy_ce + sell_ce) + (sell_pe + buy_pe))
        points_collected = points_collected_tuesday + adjusted_price
        print ("-------------------")
        print ("Total Points   : {:.2f} <=".format(points_collected))

    #Go to the Next Day - Which is Wednesday
    next_day_tuesday_datetime = pd.to_datetime(next_day_tuesday)
    next_day_wednesday = (next_day_tuesday_datetime + timedelta(days=1)).strftime("%d-%b-%y")

    #Extract the Closing Price of the CE & PE (Tuesday)
    ce_buy_close_price_wednesday = np.float(df_ce[(df_ce['Strike Price'] == (strike_price_ce+300)) & (df_ce['Date'] == next_day_wednesday) & (df_ce['Expiry'] == expiry_thursday)]['Close'])
    ce_sell_close_price_wednesday = np.float(df_ce[(df_ce['Strike Price'] == strike_price_ce) & (df_ce['Date'] == next_day_wednesday) & (df_ce['Expiry'] == expiry_thursday)]['Close'])
    pe_sell_close_price_wednesday = np.float(df_pe[(df_pe['Strike Price'] == new_strike_price_pe) & (df_pe['Date'] == next_day_wednesday) & (df_pe['Expiry'] == expiry_thursday)]['Close'])
    pe_buy_close_price_wednesday = np.float(df_pe[(df_pe['Strike Price'] == (strike_price_pe-300)) & (df_pe['Date'] == next_day_wednesday) & (df_pe['Expiry'] == expiry_thursday)]['Close'])

    print (" -------------------")
    print ("|WEDNESDAY|{}|".format(next_day_wednesday))
    print (" -------------------")
    print ("Buy - {}CE : {:.1f} ".format((strike_price_ce+300),ce_buy_close_price_wednesday))
    print ("Sell- {}CE : {:.1f} ".format(strike_price_ce,ce_sell_close_price_wednesday))
    print ("Sell- {}PE : {:.1f} ".format(new_strike_price_pe,pe_sell_close_price_wednesday))
    print ("Buy - {}PE : {:.1f} ".format((strike_price_pe-300),pe_buy_close_price_wednesday))

    
    buy_ce = (ce_buy_close_price_wednesday - ce_buy_close_price_monday)
    sell_ce = (ce_sell_close_price_monday - ce_sell_close_price_wednesday)
    if (adjust_ind):
        sell_pe = (pe_sell_close_price_tuesday - pe_sell_close_price_wednesday)
    else:
        sell_pe = (pe_sell_close_price_monday - pe_sell_close_price_wednesday)
    buy_pe = (pe_buy_close_price_wednesday - pe_buy_close_price_monday)

    points_collected_wednesday = ((buy_ce + sell_ce) + (sell_pe + buy_pe))
    points_collected = points_collected_wednesday + adjusted_price
    
    print ("-------------------")
    print ("Total Points  : {:.1f} <==|".format(points_collected))
    print (" -------------------")
    print ("|FINAL POINT: {:.1f}|".format(points_collected))
    print ("|FINAL PROFIT: {:.0f}|".format((points_collected*1600)))
    profit_pct = ((points_collected*1600)/2500000)*100
    print ("|PROFIT %: {}|".format(profit_pct))
    print (" -------------------")

#----------------------------------------
#Load the Data, Do No Run Multiple Times
#----------------------------------------
df_bank,df_ce,df_pe = load_data()

#Main Function to call the Function
trade_now('28-Dec-20')